In [1]:
import pandas as pd

In [2]:
cols = pd.read_csv("../Resources/Columns.csv")
data = pd.read_csv("../Resources/nhes_19_pfi_v1_0.csv")

col_list = []

In [3]:
# Load column names to list from CSV
for col, row in cols.iteritems():
    col_list = row.values

col_list

array(['ALLGRADEX', 'SEGRADES', 'CENREG', 'EDCPUB', 'EDCCAT', 'EDCREL',
       'EDCPRI', 'EDCINTK12', 'EDCHSFL', 'DISTASSI', 'SCHRTSCHL',
       'SCHLMAGNET', 'SPBSCH ', 'SOTHRSCH ', 'STUTR ', 'SOTHSCH ',
       'SEENJOY', 'SEABSNT', 'FCSCHOOL', 'FCTEACHR', 'FCSTDS',
       'FCSUPPRT ', 'FHHOME', 'FHWKHRS', 'FOSTORY2X', 'FOCRAFTS',
       'FOGAMES', 'FOBUILDX', 'FOSPORT', 'FORESPON', 'FOHISTX',
       'FODINNERX', 'FOLIBRAYX', 'FOBOOKSTX', 'FOCONCRTX', 'FOMUSEUMX',
       'FOZOOX', 'FOGROUPX', 'FOSPRTEVX ', 'HHENGLISH', 'CSPEAKX',
       'HHTOTALXX', 'HHPRTNRSX', 'P1REL', 'P1SEX', 'P1MRSTA ', 'P1AGE',
       'P2GUARD ', 'P2AGE', 'P2REL', 'P2SEX', 'P2MRSTA', 'PAR1EMPL',
       'PAR1FTFY', 'PAR2FTFY', 'NUMSIBSX', 'TTLHHINC', 'OWNRNTHB',
       'HVINTSPHO', 'HVINTCOM', 'INTACC', 'CHLDNT', 'LRNCOMP', 'LRNTAB',
       'LRNCELL'], dtype=object)

In [4]:
# Filter columns by new list
new_df = data.filter(col_list)
new_df

,ALLGRADEX,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,DISTASSI,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,12,1,4,1,2,2,2,2,2,1,...,1,8,2,1,1,1,1,1,2,1
1,4,5,4,1,2,2,2,2,2,1,...,1,5,2,1,1,1,1,1,1,1
2,11,3,2,2,2,1,2,2,2,-1,...,1,3,2,1,2,3,4,1,2,1
3,6,1,2,1,2,2,2,2,2,2,...,1,2,1,1,1,1,2,1,1,1
4,9,1,2,1,2,2,2,2,2,1,...,1,10,1,1,1,1,2,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16441,7,1,3,1,2,2,2,2,2,1,...,2,9,1,1,1,1,2,1,1,1
16442,2,5,2,1,2,2,2,2,2,1,...,3,9,1,1,1,1,1,2,1,2
16443,12,1,1,1,2,2,2,2,2,1,...,2,2,2,1,1,1,1,1,2,1
16444,13,1,4,1,2,2,2,2,2,2,...,0,3,2,1,2,3,2,2,2,1


In [5]:
# Remove rows with -1 or -9 as a value
values = [-1, -9]
grades = [5]
columns = list(new_df)
print(columns)

for col in columns:
    if (col == "SEGRADES"):
        new_df = new_df[new_df[col].isin(grades) == False]
    else:
        new_df = new_df[new_df[col].isin(values) == False]
new_df

['ALLGRADEX', 'SEGRADES', 'CENREG', 'EDCPUB', 'EDCCAT', 'EDCREL', 'EDCPRI', 'EDCINTK12', 'EDCHSFL', 'DISTASSI', 'SCHRTSCHL', 'SCHLMAGNET', 'SEENJOY', 'SEABSNT', 'FCSCHOOL', 'FCTEACHR', 'FCSTDS', 'FHHOME', 'FHWKHRS', 'FOSTORY2X', 'FOCRAFTS', 'FOGAMES', 'FOBUILDX', 'FOSPORT', 'FORESPON', 'FOHISTX', 'FODINNERX', 'FOLIBRAYX', 'FOBOOKSTX', 'FOCONCRTX', 'FOMUSEUMX', 'FOZOOX', 'FOGROUPX', 'HHENGLISH', 'CSPEAKX', 'HHTOTALXX', 'HHPRTNRSX', 'P1REL', 'P1SEX', 'P1AGE', 'P2AGE', 'P2REL', 'P2SEX', 'P2MRSTA', 'PAR1EMPL', 'PAR1FTFY', 'PAR2FTFY', 'NUMSIBSX', 'TTLHHINC', 'OWNRNTHB', 'HVINTSPHO', 'HVINTCOM', 'INTACC', 'CHLDNT', 'LRNCOMP', 'LRNTAB', 'LRNCELL']


,ALLGRADEX,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,DISTASSI,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,12,1,4,1,2,2,2,2,2,1,...,1,8,2,1,1,1,1,1,2,1
4,9,1,2,1,2,2,2,2,2,1,...,1,10,1,1,1,1,2,1,2,2
7,8,1,2,1,2,2,2,2,2,1,...,2,3,1,1,1,1,1,1,1,1
9,9,1,3,1,2,2,2,2,2,2,...,1,11,1,1,1,1,1,1,1,2
11,15,2,3,1,2,2,2,2,2,2,...,2,3,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16438,12,1,1,1,2,2,2,2,2,2,...,1,8,1,1,1,1,1,1,1,1
16439,14,1,1,1,2,2,2,2,2,1,...,2,4,2,1,1,1,1,1,1,1
16441,7,1,3,1,2,2,2,2,2,1,...,2,9,1,1,1,1,2,1,1,1
16443,12,1,1,1,2,2,2,2,2,1,...,2,2,2,1,1,1,1,1,2,1


In [6]:
# Check for rows with -1 as a value
count = new_df[new_df.columns[1:]].eq(-1).sum().to_frame().T
for col, row in count.iteritems():
    print(col)
    print(row.values)

SEGRADES
[0]
CENREG
[0]
EDCPUB
[0]
EDCCAT
[0]
EDCREL
[0]
EDCPRI
[0]
EDCINTK12
[0]
EDCHSFL
[0]
DISTASSI
[0]
SCHRTSCHL
[0]
SCHLMAGNET
[0]
SEENJOY
[0]
SEABSNT
[0]
FCSCHOOL
[0]
FCTEACHR
[0]
FCSTDS
[0]
FHHOME
[0]
FHWKHRS
[0]
FOSTORY2X
[0]
FOCRAFTS
[0]
FOGAMES
[0]
FOBUILDX
[0]
FOSPORT
[0]
FORESPON
[0]
FOHISTX
[0]
FODINNERX
[0]
FOLIBRAYX
[0]
FOBOOKSTX
[0]
FOCONCRTX
[0]
FOMUSEUMX
[0]
FOZOOX
[0]
FOGROUPX
[0]
HHENGLISH
[0]
CSPEAKX
[0]
HHTOTALXX
[0]
HHPRTNRSX
[0]
P1REL
[0]
P1SEX
[0]
P1AGE
[0]
P2AGE
[0]
P2REL
[0]
P2SEX
[0]
P2MRSTA
[0]
PAR1EMPL
[0]
PAR1FTFY
[0]
PAR2FTFY
[0]
NUMSIBSX
[0]
TTLHHINC
[0]
OWNRNTHB
[0]
HVINTSPHO
[0]
HVINTCOM
[0]
INTACC
[0]
CHLDNT
[0]
LRNCOMP
[0]
LRNTAB
[0]
LRNCELL
[0]


In [7]:
len(new_df)
new_df

,ALLGRADEX,SEGRADES,CENREG,EDCPUB,EDCCAT,EDCREL,EDCPRI,EDCINTK12,EDCHSFL,DISTASSI,...,NUMSIBSX,TTLHHINC,OWNRNTHB,HVINTSPHO,HVINTCOM,INTACC,CHLDNT,LRNCOMP,LRNTAB,LRNCELL
0,12,1,4,1,2,2,2,2,2,1,...,1,8,2,1,1,1,1,1,2,1
4,9,1,2,1,2,2,2,2,2,1,...,1,10,1,1,1,1,2,1,2,2
7,8,1,2,1,2,2,2,2,2,1,...,2,3,1,1,1,1,1,1,1,1
9,9,1,3,1,2,2,2,2,2,2,...,1,11,1,1,1,1,1,1,1,2
11,15,2,3,1,2,2,2,2,2,2,...,2,3,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16438,12,1,1,1,2,2,2,2,2,2,...,1,8,1,1,1,1,1,1,1,1
16439,14,1,1,1,2,2,2,2,2,1,...,2,4,2,1,1,1,1,1,1,1
16441,7,1,3,1,2,2,2,2,2,1,...,2,9,1,1,1,1,2,1,1,1
16443,12,1,1,1,2,2,2,2,2,1,...,2,2,2,1,1,1,1,1,2,1


In [ ]:
# Consilidate grades into pass or fail
passing = [1]
failing = [2,3,4]

for index, row in new_df.iterrows():
    if row[1] == passing[0]:
        row[1] = 1
    if row[1] == failing[0]:
        row[1] = 0
    elif row[1] == failing[1]:
        row[1] = 0
    elif row[1] == failing[2]:
        row[1] = 0
        
# Consilidate P1AGE ahd P2AGE
#range_1 = range(0, 20, 1)
#range_2 = range(20, 40, 1)
#range_3 = range(40, 60, 1)
#range_4 = range(60, 80, 1)
#range_5 = range(80, 100, 1)

# P1AGE
#for index, row in new_df.iterrows():
    #if row[39] in range_1:
        #row[39] = 1
    #if row[39] in range_2:
        #row[39] = 2
    #if row[39] in range_3:
        #row[39] = 3
    #if row[39] in range_4:
        #row[39] = 4
    #if row[39] in range_5:
        #row[39] = 5
# P2AGE
#for index, row in new_df.iterrows():
    #if row[40] in range_1:
        #row[40] = 1
    #if row[40] in range_2:
        #row[40] = 2
    #if row[40] in range_3:
        #row[40] = 3
    #if row[40] in range_4:
        #row[40] = 4
    #if row[40] in range_5:
        #row[40] = 5

# Consolidate FHWKHRS
#range_1 = range(0, 20, 1)
#range_2 = range(20, 40, 1)
#range_3 = range(40, 60, 1)
#range_4 = range(60, 80, 1)

#for index, row in new_df.iterrows():
    #if row[18] in range_1:
       #row[18] = 1
    #if row[18] in range_2:
        #row[18] = 2
    #if row[18] in range_3:
        #row[18] = 3
    #if row[18] in range_4:
        #row[18] = 4
        
# Consolidate FODINNERX
#for index, row in new_df.iterrows():
    #if row[18] <= 0 <= 2:
        row[18] = 1
    if row[18] <= 3 <= 4:
        row[18] = 2
    if row[18] <= 5 <= 6:
        row[18] = 3
    if row[18] >= 6:
        row[18] = 4

In [ ]:
kept_col = []
for col in new_df.columns:
    kept_col.append(col)
kept_col

In [ ]:
new_df

In [ ]:
loans_binary_encoded = pd.get_dummies(new_df, columns=kept_col)

In [ ]:
loans_binary_encoded.to_csv("../Resources/nhes_19_pfi_cleaned.csv")